In [54]:
import numpy as np, pandas as pd
from matplotlib.pyplot import subplots
from sklearn.linear_model import \
     (LinearRegression,
      LogisticRegression,
      Lasso)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from sklearn.model_selection import \
     (train_test_split,
      GridSearchCV)

import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset

from torchmetrics import (MeanAbsoluteError,
                          R2Score)
from torchinfo import summary

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger

from pytorch_lightning import seed_everything
seed_everything(0, workers=True)
torch.use_deterministic_algorithms(True, warn_only=True)

from ISLP.torch import (SimpleDataModule,
                        SimpleModule,
                        ErrorTracker,
                        rec_num_workers)


Seed set to 0


In [55]:
NYSE = load_data('NYSE')
cols = ['DJ_return', 'log_volume', 'log_volatility']
print(NYSE.columns)



Index(['day_of_week', 'DJ_return', 'log_volume', 'log_volatility', 'train'], dtype='object')


In [56]:
# Standardize the features (z-score normalization)
X = pd.DataFrame(StandardScaler(
                     with_mean=True,
                     with_std=True).fit_transform(NYSE[cols]),
                 columns=NYSE[cols].columns,
                 index=NYSE.index)

# Create a lagged data
for lag in range(1, 6):
    for col in cols:
        newcol = np.zeros(X.shape[0]) * np.nan
        newcol[lag:] = X[col].values[:-lag]
        X.insert(len(X.columns), "{0}_{1}".format(col, lag), newcol)
X.insert(len(X.columns), 'train', NYSE['train'])
X = X.dropna()

print(X.columns)

Index(['DJ_return', 'log_volume', 'log_volatility', 'DJ_return_1',
       'log_volume_1', 'log_volatility_1', 'DJ_return_2', 'log_volume_2',
       'log_volatility_2', 'DJ_return_3', 'log_volume_3', 'log_volatility_3',
       'DJ_return_4', 'log_volume_4', 'log_volatility_4', 'DJ_return_5',
       'log_volume_5', 'log_volatility_5', 'train'],
      dtype='object')


In [57]:
# extract the response and features
Y, train = X['log_volume'], X['train']
X = X.drop(columns=['train'] + cols)


In [58]:
M = LinearRegression() # create a linear regression model
M.fit(X[train], Y[train]) # fit the model on training data
M.score(X[~train], Y[~train]) # evaluate the model on test data

0.4128912938562521

In [59]:
X_day = pd.merge(X, 
                 pd.get_dummies(NYSE['day_of_week']),
                 on='date') # add features for day of the week: mon, tues, wed, thu, fri
                            # encode to dummies (categorical to binary features)

X_day.columns

Index(['DJ_return_1', 'log_volume_1', 'log_volatility_1', 'DJ_return_2',
       'log_volume_2', 'log_volatility_2', 'DJ_return_3', 'log_volume_3',
       'log_volatility_3', 'DJ_return_4', 'log_volume_4', 'log_volatility_4',
       'DJ_return_5', 'log_volume_5', 'log_volatility_5', 'mon', 'tues', 'wed',
       'thur', 'fri'],
      dtype='object')

In [60]:
M.fit(X_day[train], Y[train]) # fit the model on training data
M.score(X_day[~train], Y[~train]) # evaluate the model on test data

0.4595563133053274

## Fit the model using RNN

In [61]:
# reverse the the order of columns to have lag 5 first, then lag 4, ..., lag 1
ordered_cols = []
for lag in range(5,0,-1):
    for col in cols:
        ordered_cols.append('{0}_{1}'.format(col, lag))
X = X.reindex(columns=ordered_cols)
X.columns
X.head()


,DJ_return_5,log_volume_5,log_volatility_5,DJ_return_4,log_volume_4,log_volatility_4,DJ_return_3,log_volume_3,log_volatility_3,DJ_return_2,log_volume_2,log_volatility_2,DJ_return_1,log_volume_1,log_volatility_1
date,,,,,,,,,,,,,,,
1962-12-10,-0.549823,0.175075,-4.357078,0.905200,1.517291,-2.529058,0.434813,2.283789,-2.418037,-0.431397,0.935176,-2.366521,0.046340,0.224779,-2.500970
1962-12-11,0.905200,1.517291,-2.529058,0.434813,2.283789,-2.418037,-0.431397,0.935176,-2.366521,0.046340,0.224779,-2.500970,-1.304126,0.605918,-1.366028
1962-12-12,0.434813,2.283789,-2.418037,-0.431397,0.935176,-2.366521,0.046340,0.224779,-2.500970,-1.304126,0.605918,-1.366028,-0.006294,-0.013661,-1.505667
1962-12-13,-0.431397,0.935176,-2.366521,0.046340,0.224779,-2.500970,-1.304126,0.605918,-1.366028,-0.006294,-0.013661,-1.505667,0.377081,0.042552,-1.551515
1962-12-14,0.046340,0.224779,-2.500970,-1.304126,0.605918,-1.366028,-0.006294,-0.013661,-1.505667,0.377081,0.042552,-1.551515,-0.411718,-0.419836,-1.597607


In [62]:
# reshape the data
# an RNN require the input in the shape of (batch_size, sequence_length, num_features)
# -1: infers the batch size automatically
# 5: sequence length (number of time steps)
# 3: number of features at each time step
X_rnn = X.to_numpy().reshape((-1,5,3))
X_rnn.shape

(6046, 5, 3)

In [64]:
class NYSEModel(nn.Module): # inherit from nn.Module
    def __init__(self):
        super(NYSEModel, self).__init__() # initialize the parent class
        self.rnn = nn.RNN(3, # input size (num_features)
                          12, # hidden size
                          batch_first=True)
        self.dense = nn.Linear(12, 1)
        self.dropout = nn.Dropout(0.1) # 10% dropout
    
    def forward(self, x):
        val, h_n = self.rnn(x)
        val = self.dense(self.dropout(val[:,-1]))
        return torch.flatten(val)

nyse_model = NYSEModel()

# Prepare the data set for pytorch
datasets = []
for mask in [train, ~ train]:
    X_rnn_t = torch.tensor(X_rnn[mask].astype(np.float32))
    Y_t = torch.tensor(Y[mask].astype(np.float32))
    datasets.append(TensorDataset(X_rnn_t, Y_t))

nyse_train, nyse_test = datasets

/tmp/ipykernel_10323/118290510.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Y_t = torch.tensor(Y[mask].astype(np.float32))
/tmp/ipykernel_10323/118290510.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Y_t = torch.tensor(Y[mask].astype(np.float32))


In [ ]:
summary(nyse_model,
        input_data=X_rnn_t,
        col_names=['input_size',
                   'output_size',
                   'num_params'])
# The RNN has:
# RNN Layer Parameters:
# An RNN has two sets of weights and biases:

# Input-to-Hidden weights: W_ih

# Shape: (hidden_size × input_size) = 12 × 3 = 36 parameters
# Hidden-to-Hidden weights: W_hh

# Shape: (hidden_size × hidden_size) = 12 × 12 = 144 parameters
# Input-to-Hidden bias: b_ih

# Shape: (hidden_size) = 12 = 12 parameters
# Hidden-to-Hidden bias: b_hh

# Shape: (hidden_size) = 12 = 12 parameters
# Total RNN parameters: 36 + 144 + 12 + 12 = 204


# the dense layer has: (12 + 1) = 13 parameters

# => Total parameters: 204 + 13 = 217 parameters

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
NYSEModel                                [1770, 5, 3]              [1770]                    --
├─RNN: 1-1                               [1770, 5, 3]              [1770, 5, 12]             204
├─Dropout: 1-2                           [1770, 12]                [1770, 12]                --
├─Linear: 1-3                            [1770, 12]                [1770, 1]                 13
Total params: 217
Trainable params: 217
Non-trainable params: 0
Total mult-adds (M): 1.83
Input size (MB): 0.11
Forward/backward pass size (MB): 0.86
Params size (MB): 0.00
Estimated Total Size (MB): 0.97

In [ ]:
max_num_workers = rec_num_workers()
print("max_num_workers:", max_num_workers)

nyse_dm = SimpleDataModule(nyse_train,
                           nyse_test,
                           num_workers=min(4, max_num_workers),
                           validation=nyse_test,
                           batch_size=64)
# validate the train_dataloader
for idx, (x, y) in enumerate(nyse_dm.train_dataloader()):
    out = nyse_model(x)
    print(y.size(), out.size()) # y.size() should be equal to out.size()
    if idx >= 2:
        break


max_num_workers: 16
torch.Size([64]) torch.Size([64])
torch.Size([64]) torch.Size([64])
torch.Size([64]) torch.Size([64])


In [ ]:
# nyse_model.parameters() => list of model parameters, 217 params
# lr = 0.001 => learning rate
nyse_optimizer = RMSprop(nyse_model.parameters(),
                         lr=0.001)
nyse_module = SimpleModule.regression(nyse_model,
                                      optimizer=nyse_optimizer,
                                      metrics={'r2':R2Score()})


In [69]:
nyse_trainer = Trainer(deterministic=True,
                       max_epochs=200,
                       callbacks=[ErrorTracker()])# init a trainer
nyse_trainer.fit(nyse_module,
                 datamodule=nyse_dm) # start the training here
nyse_trainer.test(nyse_module,
                  datamodule=nyse_dm)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params | Mode 
--------------------------------------------
0 | model | NYSEModel | 217    | train
1 | loss  | MSELoss   | 0      | train
--------------------------------------------
217       Trainable params
0         Non-trainable params
217       Total params
0.001     Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.6213075518608093
         test_r2            0.4103488326072693
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.6213075518608093, 'test_r2': 0.4103488326072693}]

# Use a flatten feedforward Neural Network

In [74]:
# Prepare the dataset
# We dont need to perform reshaping here
# dont need to  generate a lagged dataset as an RNN
# Just use the original data
datasets = []
for mask in [train, ~train]:
    X_day_t = torch.tensor(
                   np.asarray(X_day[mask]).astype(np.float32))
    Y_t = torch.tensor(np.asarray(Y[mask]).astype(np.float32))
    datasets.append(TensorDataset(X_day_t, Y_t))
day_train, day_test = datasets
print(day_train.tensors[0].shape)

torch.Size([4276, 20])


In [75]:
# Create a DataModule

day_dm = SimpleDataModule(day_train,
                          day_test,
                          num_workers=min(4, max_num_workers),
                          validation=day_test,
                          batch_size=64)


Build a nonlinearARModel

In [ ]:
class NonLinearARModel(nn.Module):
    def __init__(self):
        super(NonLinearARModel, self).__init__()
        self._forward = nn.Sequential(nn.Flatten(),
                                      nn.Linear(20, 32),
                                      nn.ReLU(),
                                      nn.Dropout(0.5),
                                      nn.Linear(32, 1))
    def forward(self, x):
        return torch.flatten(self._forward(x))


Init model and optimizer and start the training

In [ ]:
nl_model = NonLinearARModel()
nl_optimizer = RMSprop(nl_model.parameters(),
                           lr=0.001)
nl_module = SimpleModule.regression(nl_model,
                                        optimizer=nl_optimizer,
                                        metrics={'r2':R2Score()}) # wrapper for torch Litening module

# create a trainer
nl_trainer = Trainer(deterministic=True,
                         max_epochs=20,
                         callbacks=[ErrorTracker()])

# fit the nonlinear AR model to the data
nl_trainer.fit(nl_module, datamodule=day_dm)
nl_trainer.test(nl_module, datamodule=day_dm) 